In [ ]:
# PLC / TwinCAT parsing + KG + templates
pip install rdflib jinja2 lxml
pip install plcopen                # PLCopen XML (IEC 61131-10) parser
pip install pytwincatparser        # parse TwinCAT .TcPOU/.TcDUT/.TcPlcProj files
pip install pyads                  # ADS read/write during tests
pip install pywin32                # COM access to TwinCAT Automation Interface from Python (Windows)


1) Canonical data model (cell)

A tiny schema to unify Skills and PLC Programs, including “parameter diagram” constraints.

In [ ]:
from dataclasses import dataclass, field
from typing import List, Dict, Optional, Literal
from pathlib import Path

Direction = Literal["in","out","inout"]

@dataclass
class Param:
    name: str
    dtype: str
    direction: Direction
    default: Optional[str] = None
    unit: Optional[str] = None
    iri: Optional[str] = None              # link into KG

@dataclass
class ProgramSig:
    name: str
    pou_type: Literal["PROGRAM","FUNCTION_BLOCK","FUNCTION"]
    inputs: List[Param] = field(default_factory=list)
    outputs: List[Param] = field(default_factory=list)
    locals: List[Param]  = field(default_factory=list)
    source_path: Optional[Path] = None     # .TcPOU or PLCopen XML file
    annotations: Dict[str,str] = field(default_factory=dict)  # e.g. comments @skill:...

@dataclass
class SkillSpec:
    skill_id: str
    name: str
    params_in: List[Param]
    params_out: List[Param]
    preconditions: List[str] = field(default_factory=list)
    effects: List[str] = field(default_factory=list)
    plc_binding: Optional[str] = None      # "GVL.Skill_Start(...)" or FB instance path
    provenance: Dict[str,str] = field(default_factory=dict)   # PFMEA-MSR links (FM, CA, DA, RPN, …)

@dataclass
class Constraint:
    """SysML-like constraint eqn tying Skill params to PLC symbols (your 'parameter diagram')."""
    expr: str                               # e.g., "t_stop >= t_min"
    binds: Dict[str,str]                    # {"t_stop": "GVL.Belt.StopTime", "t_min": "KG:Param/MinStopTime"}

@dataclass
class MappingRule:
    skill_pattern: str                      # e.g. regex on skill name
    st_template: str                        # Jinja2 template to generate ST
    deploy_task: Optional[str] = None       # TwinCAT task name


2) Importers: TwinCAT & PLCopen (cell)

Read TwinCAT .TcPOU / .TcGvl (fast path) or vendor-neutral PLCopen XML.

In [ ]:
from pytwincatparser import config as tc_cfg
from pytwincatparser import Loader, get_default_strategy
from plcopen import XmlParser, XmlContext, Project as PlcopenProject
from pathlib import Path

def load_twincat_objects(paths: list[Path]):
    tc_cfg.default_strategy = "twincat4024"
    loader = Loader(loader_strategy=get_default_strategy()())
    collected = []
    for p in paths:
        collected.extend(loader.load_objects(path=str(p)))
    return collected

def parse_plcopen_xml(xml_path: Path) -> PlcopenProject:
    with open(xml_path, "rb") as f:
        return XmlParser(context=XmlContext()).from_bytes(f.read(), PlcopenProject)

def program_sigs_from_tc(objects) -> list[ProgramSig]:
    # Minimal extractor: adapt per your tcobjects fields
    sigs = []
    for o in objects:
        if o.__class__.__name__.lower().startswith(("pou","program","function","functionblock")):
            name = getattr(o, "name", None) or o.get_identifier()
            pou_type = "PROGRAM" if "program" in o.__class__.__name__.lower() else "FUNCTION_BLOCK"
            sigs.append(ProgramSig(name=name, pou_type=pou_type, source_path=None))
    return sigs


3) Translators: Program → Skill and Skill → Program (cells)

Use heuristics or annotations to lift a POU into a Skill; go back via Jinja2 ST templates.

In [ ]:
import re, jinja2

def program_to_skill(p: ProgramSig) -> SkillSpec:
    # Heuristic: function-block with Execute/Start → an invocable Skill
    params_in  = [Param(name="Start", dtype="BOOL", direction="in")] if p.pou_type=="FUNCTION_BLOCK" else []
    params_out = [Param(name="Done",  dtype="BOOL", direction="out")]
    name = p.annotations.get("skill", p.name)  # allow @skill:PickAndPlace in comments
    return SkillSpec(
        skill_id=f"skill:{name}",
        name=name,
        params_in=params_in,
        params_out=params_out,
        preconditions=[],
        effects=[],
        plc_binding=f"{p.name}"  # adjust to instance path later
    )

ST_TEMPLATE = """\
FUNCTION_BLOCK {{ fb_name }}
VAR_INPUT
{%- for p in params_in %}    {{p.name}} : {{p.dtype}};{% endfor %}
END_VAR
VAR_OUTPUT
{%- for p in params_out %}    {{p.name}} : {{p.dtype}};{% endfor %}
END_VAR
VAR
    _state : INT := 0;
END_VAR
(* Auto-generated from Skill {{skill_name}} *)
CASE _state OF
0: IF {{ start_expr }} THEN _state := 1; END_IF;
1: (* TODO: add PLC logic per mapping/constraints *) {{ done_set }};
END_CASE
END_FUNCTION_BLOCK
"""

def skill_to_program(skill: SkillSpec, rule: MappingRule) -> str:
    env = jinja2.Environment(autoescape=False, trim_blocks=True, lstrip_blocks=True)
    tmpl = env.from_string(rule.st_template or ST_TEMPLATE)
    return tmpl.render(
        fb_name=f"FB_{skill.name}",
        skill_name=skill.name,
        params_in=skill.params_in,
        params_out=skill.params_out,
        start_expr=skill.params_in[0].name if skill.params_in else "TRUE",
        done_set=f"{skill.params_out[0].name} := TRUE;" if skill.params_out else ""
    )


4) “Parameter diagram” tracking (cell)

Bind KG terms ↔ PLC symbols and enforce constraints.

In [ ]:
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, RDFS, XSD

NS = Namespace("http://example.org/kg#")

def constraint_to_triples(skill: SkillSpec, c: Constraint):
    # store as KG facts; simple shape, extend with SHACL later
    g = Graph()
    skill_uri = URIRef(f"http://example.org/skill/{skill.name}")
    constr_uri = URIRef(f"{skill_uri}/constraint/{abs(hash(c.expr))}")
    g.add((skill_uri, NS.hasConstraint, constr_uri))
    g.add((constr_uri, NS.expr, Literal(c.expr)))
    for k,v in c.binds.items():
        g.add((constr_uri, NS.bind, Literal(f"{k}={v}")))
    return g

def diff_plc_vs_kg(plc_symbols: Dict[str,str], kg_bindings: Dict[str,str]) -> dict:
    missing_in_plc = {k:v for k,v in kg_bindings.items() if v not in plc_symbols.values()}
    missing_in_kg  = {sym:t for sym,t in plc_symbols.items() if sym not in set(kg_bindings.values())}
    return {"missing_in_plc": missing_in_plc, "missing_in_kg": missing_in_kg}


5) Deploy to TwinCAT (Automation Interface) (cell)

Minimal Python COM snippet to open VS/XAE, import PLCopen XML, build, activate. (Run on your engineering PC with TwinCAT XAE installed.)

In [ ]:
import win32com.client as com
import time, os

def tc_ai_open_solution(sol_path:str):
    # open Visual Studio + XAE via DTE then talk to TcS+ysManager (progIDs vary by VS version)
    dte = com.Dispatch("VisualStudio.DTE.17.0")    # VS2019 example; adjust if 17.0 (VS2022)
    dte.UserControl = True
    dte.MainWindow.Visible = True
    dte.Solution.Open(sol_path)
    # TwinCAT System Manager COM
    tcsys = com.Dispatch("TcXaeShell.TcSysManager")   # on recent TwinCAT/XAE; see manual table
    return dte, tcsys

def tc_ai_import_plcopen(tcsys, plcproj_guid:str, plcopen_path:str):
    # navigate to your PLC project node and import IEC 61131-10 XML
    # (in practice: use TCatSysManagerLib interfaces; sample is indicative)
    root = tcsys.ActiveTargetProject   # pseudo; inspect object model on your machine
    plc = root.LookupTreeItem(f"TIPC^PLC^{plcproj_guid}")
    plc.Import(plcopen_path)           # typical Automation Interface method
    return plc

def tc_ai_build_activate(tcsys, ams_net_id:str):
    tcsys.ActivateConfiguration()
    time.sleep(3)
    tcsys.StartRestartTwinCAT(ams_net_id)   # switch to RUN


6) Quick ADS smoke test (cell)

In [ ]:
import pyads

def ads_check_bool(ams:str, varname:str):
    with pyads.Connection(ams, pyads.PORT_TC3PLC1) as plc:
        return plc.read_by_name(varname, pyads.PLCTYPE_BOOL)

7) End-to-end flow (cell)

Glue it together for Programs→Skills→ST→PLCopen→TwinCAT and back.

In [ ]:
from pathlib import Path
from rdflib import Graph

# 1) parse TwinCAT or PLCopen
tc_objects = load_twincat_objects([Path(r"D:\…\YourProject\Plc\*.TcPOU")])
progs      = program_sigs_from_tc(tc_objects)

# 2) lift program to skill
skills = [program_to_skill(p) for p in progs if "Diag" in p.name]  # e.g., diagnosis POUs → Skills

# 3) add a constraint (your parameter diagram binding)
c = Constraint(expr="t_stop >= t_min", binds={"t_stop":"GVL.Belt.StopTime", "t_min":"KG:/params/MinStopTime"})
g = Graph()
for s in skills: g += constraint_to_triples(s, c)
g.serialize("skill_constraints.ttl")

# 4) generate ST from Skill
rule = MappingRule(skill_pattern=".*", st_template=ST_TEMPLATE, deploy_task="PlcTask")
for s in skills:
    st_code = skill_to_program(s, rule)
    Path(f"gen_FB_{s.name}.st").write_text(st_code, encoding="utf-8")

# 5) create PLCopen XML from ST (simple wrap or use your exporter)
#    (In practice: add ST into your .TcPOU via Automation Interface or export PLCopen XML and Import)
# 6) import/build/activate on TwinCAT
# dte, tcsys = tc_ai_open_solution(r"D:\…\YourTwinCAT.sln")
# plc = tc_ai_import_plcopen(tcsys, "{PLC-GUID-HERE}", r"D:\…\generated\DiagSkills.xml")
# tc_ai_build_activate(tcsys, "5.38.17.123.1.1")  # <- your AMS Net ID

# 7) ADS poke
# ok = ads_check_bool("5.38.17.123.1.1", "MAIN.FB_DiagSkill.Done")
